In [21]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
import plotnine as gg
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
topdir="Performance"
subdir='W47_2020'
figures = "figures"

## Directories and Filepath

In [23]:
def paths(topdir, subdir, figures):
    
    folderpath = os.path.join(Path("../", topdir), subdir)

    figpath = os.path.join(folderpath, figures)


    def create_folder(path):
             
        if not os.path.exists(path):
        
            print ("directory is created")
        
            return os.makedirs(path)
    
        else:
            
            return print ("directory is already existed")

    
    
    folder = create_folder(folderpath)

    figdir = create_folder(figpath)
    
    return folderpath, figpath


folderpath, figpath = paths(topdir, subdir, figures)
print(folderpath)
print(figpath)

directory is already existed
directory is already existed
../Performance/W47_2020
../Performance/W47_2020/figures


## Defining Paths

In [24]:
def csv_paths(folderpath):
    
    csvpath = [os.path.join(folderpath, f) for f in os.listdir(folderpath)]
    
    apriso_path = [f for f in csvpath if "Apriso" in f][0]
    
    tracker_path = [f for f in csvpath if "Tracker" in f][0]
    
    buffer_path = [f for f in csvpath if "Buffer" in f][0]
    
    return apriso_path, tracker_path, buffer_path


apriso_path, tracker_path, buffer_path = csv_paths(folderpath)

print(apriso_path)
print(tracker_path)
print(buffer_path)



../Performance/W47_2020/Apriso_NOV22.csv
../Performance/W47_2020/Tracker_NOV22.csv
../Performance/W47_2020/Repair Department Buffer Report.csv


### Creating Merged dataframe which resulted after matching Tracker and Apriso Data

In [ ]:
def create_merged_csv(apriso_path, tracker_path, figpath):
    
    tracker_columns = ['DATE','RBU 6701_SN', 'FAIL Category', 'Fail Group', 
                       'TRSH NAME','STATUS', 'Notes', 'REPAIR Person',
                       'REPAIR NOTES', 'PRE-TRSH Person']

    data_tracker = (pd.read_csv(tracker_path, engine='python')
                    .loc[:, tracker_columns]
                   )
    
    
    apriso_columns = ['Serial #', 'Level 1', 'Level 2', 'Level 3', 'ProductNo']
    
    data_apriso = (pd.read_csv(apriso_path)
                   .loc[:, apriso_columns]
                  )


    data_apriso['RBU 6701_SN'] = data_apriso[data_apriso.columns[0:4]].apply(
        lambda x: ''.join(x.dropna().astype(str)),
        axis=1
    )
    
    data_apriso = (data_apriso.loc[:, ['RBU 6701_SN', 'ProductNo']])
    
    merged = data_tracker.merge(data_apriso, how='left', left_on='RBU 6701_SN', right_on= 'RBU 6701_SN')

    merged = merged.drop_duplicates()

        
    merged.loc[(merged['ProductNo'] == 'ROA1286713/1') & (merged['FAIL Category'] == 'RF'), 'FAIL Category'] = "MBFT"
    
    
    merged['RBU 6701_SN'] = merged['RBU 6701_SN'].str.replace("ea2a", "EA2A")
    
    
    merged['RBU 6701_SN'] = merged['RBU 6701_SN'].str.replace("e23", "E23")
    
    return merged

merged = create_merged_csv(apriso_path, tracker_path, figpath)

merged.head()

/Users/habbasi/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (3,9) have mixed types.Specify dtype option on import or set low_memory=False.


# Repair department

## Functions

This function is for renaming of Failure Categories for all departments

In [ ]:

def repair_dept_namechange(x):
    
    if (x['FAIL Category'] == 'RF') | (x['FAIL Category'] == 'DIG.'):
        return "Radio"
    
    elif (x['FAIL Category'] == 'MBFT'):
        return "MBFT"
    
    elif (x['FAIL Category'] == 'MVI'):
        return "MVI"
    
    elif (x['FAIL Category'] == 'SMA Repair') | (x['FAIL Category'] == 'SMA REPAIR'):
        return "SMA Repair"
    
    elif (x['FAIL Category'] == 'I.C.O. (TRSH)'):
        return "I.C.O. (TRSH)"
    
    elif (x['FAIL Category'] ==  'Re-work Order'):
        return "Re-work Order"
    
    elif (x['FAIL Category'] ==  "Mech."):
        return "Mech."
    
    elif (x['FAIL Category'] == 'Validation Radio Test'):
        return 'Validation Radio Test'
    
    
    
    
TRSH_NAME=['Bo B.','Maged Y.','Matthew T.','Mekedes W.',
           'Michael Smith U.','Omar V.','Samuel T.',
           'Sujin K.','Yohannes A.']


REPAIR_NAME=['Daniel S.','Khaliah T.','Nelson L.',
             'Rachel E.','Tamela B.','Thai L.', 'Tri N.']


PRETRSH_NAME=['Alton W.','Eric T.','Kazeem K.','Luis G.','Scott N.','Ve']


MVI_NAME=['Phat N.','Tri N.','Tuan L.']


Ist_shift = ['Bo B.','Maged Y.','Matthew T.','Michael Smith U.',
             'Sujin K.','Yohannes A.', 'Daniel S.','Khaliah T.',
             'Rachel E.', 'Alton W.','Kazeem K.','Scott N.', 
             'Phat N.','Tuan L.']


Sec_shift = ['Mekedes W.','Omar V.','Samuel T.','Nelson L.',
             'Tamela B.','Thai L.', 'Tri N.', 'Eric T.',
             'Luis G.','Ve']


    



## TRSH Performance and KPI

In [ ]:
def TRSH_Performance_KPI(merged, figpath):
    
    cat_condition=["DIG.", "RF", "MBFT", "I.C.O. (TRSH)"]

    merged_df = (merged[merged['FAIL Category'].isin(cat_condition)]
                .groupby(['TRSH NAME','ProductNo', "FAIL Category"])
                .count()
                .reset_index()
                .groupby(['TRSH NAME', "FAIL Category"])
                .sum()
                .reset_index()
                .loc[:, ['TRSH NAME','FAIL Category', 'DATE']]
                .rename(columns={'TRSH NAME': 'Names',
                                 'DATE': "Count",
                                 'FAIL Category': "Category"
                                })

                )

    tmp = (merged[merged['FAIL Category'].isin(cat_condition)]
                .groupby(['TRSH NAME',"FAIL Category"])
                .count()
                .reset_index()
                .groupby(['TRSH NAME', "FAIL Category"])
                .sum()
                .reset_index()
                .loc[:, ['TRSH NAME','FAIL Category', 'DATE']]
                .rename(columns={'TRSH NAME': 'Names',
                                 'DATE': "Count",
                                 'FAIL Category': "Category"
                                })

                )

    ICO_TRSH = tmp[tmp['Category'] == "I.C.O. (TRSH)"]

    merged_df = merged_df.append(ICO_TRSH).reset_index().drop(columns='index')



    total = (merged.groupby('TRSH NAME').count()
            .reset_index()
            .loc[:, ['TRSH NAME','DATE']]
            .rename(columns={'TRSH NAME': 'Names',
                             'DATE': "Count",
                            })
            )



    total['Category'] = 'Total Attempts'

    total = total.loc[:, ['Names', 'Category', 'Count']]
    fn = merged_df.append(total)



    p = gg.ggplot(fn, gg.aes(x='Names', y='Count',fill="Category")) + \
        gg.geom_col(stat="identity") + \
        gg.theme_bw() + \
        gg.coord_flip() + \
        gg.geom_text(gg.aes(y = "Count",label = "Count"),  
                     size=8, nudge_x=0, nudge_y=2, show_legend=True) + \
        gg.labs(title= 'Troubleshooters Report' + ' (' + str(subdir) + ')')



    p = p + gg.facet_wrap('~ Category', nrow=2)



    #print(p + facet_wrap('~ mask', nrow=2, scales="free_y"))




    gg.ggsave(filename='Trouble_shooters_performance_' + str(subdir) +'.png', plot = p, path = figpath)
    
    
    ## Calculating TRSH KPI
    
    
    min_threshold_radio=25
    target_radio=35
    min_threshold_board=40
    target_board=50



    def repair_dept_kpi(x):

        if (x['Category'] == 'RF') | (x['Category'] == 'DIG.'):
            return "Radio"

        elif (x['Category'] == 'MBFT') | (x['Category'] == "I.C.O. (TRSH)"):
            return "Board"


    

    def KPI_radio(x):

        if (x['Category'] == "Radio") & (x['Names'] == 'Matthew T.'):

            k = (x['Count'] - min_threshold_radio) / (target_radio - min_threshold_radio)

            return k * 0.3

        elif (x['Category'] == "Radio") & (x['Names'] == 'Omar V.'):

            k = (x['Count'] - min_threshold_radio) / (target_radio - min_threshold_radio)

            return k * 0.7

        elif (x['Category'] == "Radio"):

            k = (x['Count'] - min_threshold_radio) / (target_radio - min_threshold_radio)

            return k 


    def KPI_board(x):

        if (x['Category'] == "Board") & (x['Names'] == 'Matthew T.'):

            k = (x['Count'] - min_threshold_board) / (target_board - min_threshold_board)

            return k * 0.4

        elif (x['Category'] == "Board") & (x['Names'] == 'Omar V.'):

            k = (x['Count'] - min_threshold_board) / (target_board - min_threshold_board)

            return k * 0.7

        elif (x['Category'] == "Board"):

            k = (x['Count'] - min_threshold_board) / (target_board - min_threshold_board)

            return k


    
        
    kpi = merged_df.copy()


    kpi['Category'] = kpi.apply(repair_dept_kpi, axis=1)


    kpi = (kpi.groupby(["Names", "Category"])
           .sum()
           .reset_index()

          )



    kpi['KPI_Radio'] = kpi.apply(KPI_radio, axis=1)



    kpi['KPI_board'] = kpi.apply(KPI_board, axis=1)





    kpi = (kpi.groupby("Names")
           .sum()
           .reset_index()
           .assign(Total_Kpi = lambda x: np.round(x['KPI_Radio'] + x['KPI_board'], 1))
          )




    g = gg.ggplot(kpi, gg.aes(x='Names', y='Total_Kpi',fill="Names")) + \
        gg.geom_col(stat="identity") + \
        gg.theme_classic() + \
        gg.geom_text(gg.aes(y = "Total_Kpi",label = "Total_Kpi"),  
                     size=8, nudge_x=0, nudge_y=0.5, show_legend=True) + \
        gg.labs(title= 'Troubleshooters KPI Report'+ ' (' + str(subdir) + ')',
                x=str(subdir),
                y="KPI Index"
               ) + \
        gg.theme(axis_text_x=gg.element_blank())


    gg.ggsave(filename='Trouble_shooters_KPI.png', plot = g, path = figpath)





    return p, g

TRSH_Performance_KPI(merged, figpath)


### Pre-TRSH performance and KPI

In [ ]:
def pre_TRSH_Performance_KPI(merged, figpath):
    
    
    
    def repair_dept_namechange(x):

        if (x['FAIL Category'] == 'RF') | (x['FAIL Category'] == 'DIG.'):
            return "Radio"

        elif (x['FAIL Category'] == 'MBFT'):
            return "MBFT"

        elif (x['FAIL Category'] == 'MVI'):
            return "MVI"

        elif (x['FAIL Category'] == 'SMA Repair') | (x['FAIL Category'] == 'SMA REPAIR'):
            return "SMA Repair"

        elif (x['FAIL Category'] == 'I.C.O. (TRSH)'):
            return "I.C.O. (TRSH)"

        elif (x['FAIL Category'] ==  'Re-work Order'):
            return "Re-work Order"

        elif (x['FAIL Category'] ==  "Mech."):
            return "Mech."

        elif (x['FAIL Category'] == 'Validation Radio Test'):
            return 'Validation Radio Test'
        
        
    
    PRETRSH_NAME=['Alton W.','Eric T.','Kazeem K.','Luis G.','Scott N.','Ve']
 
    

    excluded_trb=['Phat N.', 'Tuan L.']


    merged_trb = merged.copy()

    trb = ['DIG.', 'RF','Mech.', 'ALT2', 'Validation Radio Test', 'Re-work Order']

    merged_trb = merged_trb.loc[merged_trb['FAIL Category'].isin(trb)]

    ## Calling RF and DIG. as Radio and count twice for similar person

    merged_trb['FAIL Category'] = merged_trb.apply(repair_dept_namechange, axis=1)

    ## Dropping duplicate for pretrouble shooters

    merged_trb = (merged_trb.drop_duplicates(['RBU 6701_SN',"FAIL Category",'PRE-TRSH Person'], keep='last')
    )


    merged_trb = (merged_trb.groupby(['PRE-TRSH Person',"FAIL Category"])
                  .count()
                  .reset_index()
                  .loc[:, ['PRE-TRSH Person','FAIL Category', 'DATE']]
                  .rename(columns={'PRE-TRSH Person': 'Names',
                                   'FAIL Category':"Category",
                                   'DATE': "Count"})

                 )




    merged_trb = merged_trb[~merged_trb['Names'].isin(excluded_trb)]

    merged_trb = merged_trb[merged_trb['Names'].isin(PRETRSH_NAME)]


    pre = gg.ggplot(merged_trb, gg.aes(x='Names', y='Count', fill="Category")) + \
        gg.geom_bar(stat="identity") + \
        gg.theme_bw() + \
        gg.geom_text(gg.aes(y = "Count",label = "Count"),  
                     size=8, nudge_x=0, nudge_y=1, show_legend=False) + \
        gg.labs(title= 'Pre-Troubleshooters Report' + ' (' + str(subdir) + ')') 


    gg.ggsave(filename='Pretrouble_shooters_performance.png', plot = pre, path = figpath)
    
    
    
    min_threshold_radio=46
    target_radio=56
    min_threshold_rework=22
    target_rework=32



    def pre_TRSH_kpi(x):

        if (x['Category'] == 'Radio') | (x['Category'] == 'Mech.') | (x['Category'] == 'Validation Radio Test'):
            return "Radio"

        elif (x['Category'] == 'Re-work Order'):
            return "Rework"


    
    def KPI_preTRSH_radio(x):

        if (x['Category'] == "Radio"):

            k = (x['Count'] - min_threshold_radio) / (target_radio - min_threshold_radio)

            return k 


    def KPI_preTRSH_rework(x):

        if (x['Category'] == "Rework"):

            k = (x['Count'] - min_threshold_rework) / (target_rework- min_threshold_rework)

            return k


    
        
    KPI_preTRSH = merged_trb.copy()


    KPI_preTRSH['Category'] = KPI_preTRSH.apply(pre_TRSH_kpi, axis=1)


    KPI_preTRSH = (KPI_preTRSH.groupby(["Names", "Category"])
           .sum()
           .reset_index()

          )



    KPI_preTRSH['KPI_Radio'] = KPI_preTRSH.apply(KPI_preTRSH_radio, axis=1)



    KPI_preTRSH['KPI_Rework'] = KPI_preTRSH.apply(KPI_preTRSH_rework, axis=1)





    KPI_preTRSH = (KPI_preTRSH.groupby("Names")
           .sum()
           .reset_index()
           .assign(Total_Kpi = lambda x: np.round(x['KPI_Radio'] + x['KPI_Rework'], 1))
          )





    g = gg.ggplot(KPI_preTRSH, gg.aes(x='Names', y='Total_Kpi',fill="Names")) + \
        gg.geom_col(stat="identity") + \
        gg.theme_classic() + \
        gg.geom_text(gg.aes(y = "Total_Kpi",label = "Total_Kpi"),  
                     size=8, nudge_x=0, nudge_y=0.1, show_legend=True) + \
        gg.labs(title= 'Pre-Troubleshooters KPI Report' + ' (' + str(subdir) + ')' ,
                x=str(subdir),
                y="KPI Index"
               ) + \
        gg.theme(axis_text_x=gg.element_blank())




    gg.ggsave(filename='Pre-Trouble_shooters_KPI_'+ str(subdir) + '.png', plot = g, path = figpath)

    


    return pre, g


pre_TRSH_Performance_KPI(merged, figpath)

## MVI Performance and KPI

In [ ]:
def MVI_Performance_KPI(merged, figpath):
    
    
    def repair_dept_namechange(x):

        if (x['FAIL Category'] == 'RF') | (x['FAIL Category'] == 'DIG.'):
            return "Radio"

        elif (x['FAIL Category'] == 'MBFT'):
            return "MBFT"

        elif (x['FAIL Category'] == 'MVI'):
            return "MVI"

        elif (x['FAIL Category'] == 'SMA Repair') | (x['FAIL Category'] == 'SMA REPAIR'):
            return "SMA Repair"

        elif (x['FAIL Category'] == 'I.C.O. (TRSH)'):
            return "I.C.O. (TRSH)"

        elif (x['FAIL Category'] ==  'Re-work Order'):
            return "Re-work Order"

        elif (x['FAIL Category'] ==  "Mech."):
            return "Mech."

        elif (x['FAIL Category'] == 'Validation Radio Test'):
            return 'Validation Radio Test'
    
    
    

    mvi_trb=['Tri N.', 'Tuan L.', 'Phat N.']


    mvi = merged[merged['PRE-TRSH Person'].isin(mvi_trb)]


    mvi['FAIL Category'] = mvi.apply(repair_dept_namechange, axis=1)





    mvi = (mvi.groupby(['PRE-TRSH Person',"FAIL Category"])
           .count()
           .reset_index()
           .loc[:, ['PRE-TRSH Person','FAIL Category', 'DATE']]
           .rename(columns={'PRE-TRSH Person': 'Names',
                                   'FAIL Category':"Category",
                                   'DATE': "Count"})

                 )


    mvi_plot = gg.ggplot(mvi, gg.aes(x='Names', y='Count', fill="Category")) + \
        gg.geom_bar(stat="identity") + \
        gg.theme_bw() + \
        gg.geom_text(gg.aes(y = "Count",label = "Count"),  
                     size=8, nudge_x=0, nudge_y=0.1, show_legend=True) + \
        gg.labs(title= 'MVI Report' + ' (' + str(subdir) + ')' ) 






    gg.ggsave(filename='MVI_performance_modified_'+ str(subdir) + '.png', plot = mvi_plot, path = figpath)
    
    
    
    
    min_threshold_board_mvi=150
    
    target_board_mvi=200




    def mvi_kpi(x):

        if (x['Category'] == 'MVI') | (x['Category'] == 'SMA Repair') | (x['Category'] == 'I.C.O. (TRSH)'):
            return "board_mvi"



    def KPI_MVI(x):

        if (x['Category'] == "board_mvi"):

            k = (x['Count'] - min_threshold_board_mvi) / (target_board_mvi - min_threshold_board_mvi)

            return k 



    
        
    KPI_mvi = mvi.copy()


    KPI_mvi['Category'] = KPI_mvi.apply(mvi_kpi, axis=1)


    KPI_mvi = (KPI_mvi.groupby(["Names", "Category"])
           .sum()
           .reset_index()

          )


    KPI_mvi['KPI_MVI'] = KPI_mvi.apply(KPI_MVI, axis=1)






    mvi_kpi = gg.ggplot(KPI_mvi, gg.aes(x='Names', y='KPI_MVI',fill="Names")) + \
        gg.geom_col(stat="identity") + \
        gg.theme_classic() + \
        gg.geom_text(gg.aes(y = "KPI_MVI",label = "KPI_MVI"),  
                     size=8, nudge_x=0, nudge_y=0.1, show_legend=True) + \
        gg.labs(title= 'KPI_MVI' + ' (' + str(subdir) + ')',
                x=str(subdir),
                y="KPI Index"
               ) + \
        gg.theme(axis_text_x=gg.element_blank())



    gg.ggsave(filename='KPI_MVI_'+ str(subdir) + '.png', plot = mvi_kpi, path = figpath)
    
    return mvi_plot, mvi_kpi





MVI_Performance_KPI(merged, figpath)



## Soldering Performance and KPI

In [ ]:
def soldering_Performance_KPI(merged, figpath):
    
    
    def repairsoldering_dept_namechange(x):
    
        if (x['FAIL Category'] == 'RF') | (x['FAIL Category'] == 'DIG.'):
            return "Radio"

        elif (x['FAIL Category'] == 'SMA Repair') | (x['FAIL Category'] == 'SMA REPAIR'):
            return "SMA Repair"

        elif (x['FAIL Category'] == 'I.C.O. (TRSH)'):
            return "I.C.O. (TRSH)"

        elif (x['FAIL Category'] ==  "MBFT"):
            return "MBFT"
        
        
        
    
    REPAIR_NAME=['Daniel S.','Khaliah T.','Nelson L.',
             'Rachel E.','Tamela B.','Thai L.', 'Tri N.']



    repair = merged[merged['REPAIR Person'].isin(REPAIR_NAME)]

    repair['FAIL Category'] = (repair
                               .apply(repairsoldering_dept_namechange, axis=1)
                              )


    repair = (repair
              .drop_duplicates(['RBU 6701_SN',"FAIL Category",'REPAIR Person', 'REPAIR NOTES'], keep='last')
             )




    repair = (repair.groupby(['FAIL Category','REPAIR Person'])
              .count()
              .reset_index()
              .loc[:, ['FAIL Category', 'REPAIR Person','DATE']]
              .rename(columns={'REPAIR Person': 'Names',
                                   'FAIL Category':"Category",
                                   'DATE': "Count"})
             )



    soldering_plot = gg.ggplot(repair, gg.aes(x='Names', y='Count', fill="Category")) + \
        gg.geom_bar(stat="identity") + \
        gg.geom_text(gg.aes(y = "Count",label = "Count"),  
                     size=8, nudge_x=0, nudge_y=-1, show_legend=True) + \
        gg.labs(title= 'Soldering Report' + ' (' + str(subdir) + ')') 






    gg.ggsave(filename='soldering_performance' + str(subdir) + '.png', plot = soldering_plot, path = figpath)
    
    
    ## Calculating Soldering KPI
    
    
    min_threshold_soldering=55
    
    target_soldering=65




    soldering_KPI = (repair
                     .groupby("Names").sum().reset_index()
                    )

    

    

    soldering_KPI['Soldering KPI'] = (soldering_KPI['Count']
                                  .apply(lambda x: (x - min_threshold_soldering)/(target_soldering - min_threshold_soldering))
                                 )



        
    soldering_kpi = gg.ggplot(soldering_KPI, gg.aes(x='Names', y='Soldering KPI', fill='Names')) + \
        gg.geom_bar(stat="identity") + \
        gg.geom_text(gg.aes(y = "Soldering KPI",label = "Soldering KPI"),  
                     size=8, nudge_x=0, nudge_y=0.5, show_legend=True) + \
        gg.labs(title= 'Soldering KPI'+ ' (' + str(subdir) + ')') 


    

    

    gg.ggsave(filename='soldering_KPI' + str(subdir) + '.png', plot = soldering_kpi, path = figpath)

    
    
    return soldering_plot, soldering_kpi


soldering_Performance_KPI(merged, figpath)


 ## SCRIPT END HERE